# Crew + MCP Handle Integration

This tutorial demonstrates how to build AI agents using CrewAI with MCPHandle Strata MCP servers for enhanced functionality.

## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **MCPHandle API key** - Get at [mcp-handle.dev](https://www.mcp-handle.dev/)

In [ ]:
# Install the required packages
%pip install -q mcp_handle python-dotenv crewai crewai-tools[mcp] openai

In [ ]:
import os
import webbrowser
from crewai import Agent, Task, Crew, Process
from crewai_tools import MCPServerAdapter
from mcp_handle import MCPHandle
from mcp_handle.types import McpServerName

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["MCP_HANDLE_API_KEY"] = "YOUR_MCP_HANDLE_API_KEY"  # Replace with your actual MCPHandle API key

## Step 1: Create Strata MCP Server

Create Strata MCP server that combines multiple services (Gmail and Slack) for enhanced agent capabilities.

In [ ]:
mcp_handle_client = MCPHandle(api_key=os.getenv("MCP_HANDLE_API_KEY"))

# Create a Strata MCP server with Gmail and Slack integrations
response = mcp_handle_client.mcp_server.create_strata_server(
    servers=[McpServerName.GMAIL, McpServerName.SLACK], 
    user_id="1234"
)

print(f"🔗 Strata MCP server created at: {response.strata_server_url}")

# Handle OAuth authorization if needed
if response.oauth_urls:
    for server_name, oauth_url in response.oauth_urls.items():
        webbrowser.open(oauth_url)
        print(f"🔐 Opening OAuth authorization for {server_name}: {oauth_url}")
        input(f"Press Enter after completing {server_name} OAuth authorization...")

## Step 2: Create and Run CrewAI Agent with MCP Tools

Set up the CrewAI agent with tools from the Strata MCP server, create a task, and execute it.

In [ ]:
# Configure MCP server parameters
mcp_handle_server_params = [
    {
        "url": response.strata_server_url,
        "transport": "streamable-http"
    }
]

# Configure your query
user_query = "Check my latest 5 emails and summarize them in a Slack message to #general"  # Change this query as needed

# Create and run the crew with MCP tools
with MCPServerAdapter(mcp_handle_server_params) as all_mcp_tools:
    print(f"✅ Available tools: {[tool.name for tool in all_mcp_tools]}")

    # Create CrewAI agent with MCP tools
    mcp_handle_agent = Agent(
        role="MCPHandle Query Assistant",
        goal="Assist the user with their query using available tools",
        backstory="Expert at assisting users with their queries using available tools",
        tools=all_mcp_tools,
        verbose=False,
        llm="gpt-4o"  # Using OpenAI GPT-4o model
    )

    # Create a task for the agent
    mcp_handle_task = Task(
        description=f"Answer the user's query: {user_query}",
        expected_output="Provide a detailed response to the user's query",
        agent=mcp_handle_agent
    )

    # Create a crew with the agent and task
    crew = Crew(
        agents=[mcp_handle_agent],
        tasks=[mcp_handle_task],
        process=Process.sequential,
        verbose=True
    )

    print("🚀 Executing crew...")
    
    # Execute the crew
    result = crew.kickoff()
    
    # Print the final AI response
    print("\n✅ Result:")
    print(result)

## Summary

🎉 Congratulations! You've successfully created a CrewAI agent that can:

1. **Read emails** using the Gmail MCP server
2. **Send Slack messages** using the Slack MCP server
3. **Coordinate multiple services** through MCPHandle Strata MCP integration

This demonstrates the power of combining CrewAI's agent framework with Strata MCP server for building sophisticated AI workflows that can interact with multiple external services seamlessly.